In [13]:
import uuid
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
from langgraph.store.memory import InMemoryStore
from CONFIG import GROQ_MODEL, TEMPERATURE

In [14]:
store = InMemoryStore()

In [15]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [16]:
class pydantic_class(BaseModel):
    should_store: bool = Field(description="whether to store any memory")
    list_of_memory : List[str] = Field(default_factory=List, description="atomic user memory to store")

structured_llm = llm.with_structured_output(pydantic_class)

In [17]:
class state_class(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [18]:
def stored_LTM_memory(state: state_class, config: RunnableConfig, store: BaseStore):
    config = config['configurable']['user_id']
    namespace = ('user', config, 'details')
    last_msg = state['messages'][-1].content

    decision: pydantic_class = structured_llm.invoke(
        [
            SystemMessage(
                content=(
				"Extract LONG-TERM memories from the user's message.\n"
				"Only store stable, user-specific info (identity, preferences, ongoing projects).\n"
				"Do NOT store transient info.\n"
				"Return should_write=false if nothing is worth storing.\n"
				"Each memory should be a short atomic sentence."
                )
		),
        {'role': 'user', 'content': last_msg}
	  ]
    )


    if decision.should_store:
        for mm in decision.list_of_memory:
            store.put(namespace, str(uuid.uuid4()), {'date': mm})
            
    return {'messages': [{'role': 'assistant', 'content': 'Noted'}]}

In [19]:
builder = StateGraph(state_class)

builder.add_node('stored_LTM_memory', stored_LTM_memory)

builder.add_edge(START, 'stored_LTM_memory')
builder.add_edge('stored_LTM_memory', END)

graph = builder.compile(store=store)

In [20]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'Hi my name is adnan saeed'}]},config)
print(output['messages'][-1].content)

Noted


In [21]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'i am working as AI Engineer'}]},config)
print(output['messages'][-1].content)

Noted


In [39]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'hi'}]},config)
print(output['messages'][-1].content)

Noted


In [41]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'i want to move Dubai'}]},config)
print(output['messages'][-1].content)

Noted


In [42]:
for i in store.search(('user', 'o1', 'details')):
    print(i.value)

{'date': 'My name is Adnan Saeed'}
{'date': 'The user is working as an AI Engineer'}
{'date': 'user wants to move to Dubai'}
